<a href="https://colab.research.google.com/github/macgyver121/DADS7202_final_project/blob/main/prototypeV3_evaluate%2Bmd_k2t_24_12_65_resultOK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Check GPU

In [1]:
# List all NVIDIA GPUs as avaialble in this computer (or Colab's session)
!nvidia-smi -L

GPU 0: A100-SXM4-40GB (UUID: GPU-7af9fddc-5d06-9ee9-939d-394ed338f23b)


In [2]:
import sys
print( f"Python {sys.version}\n" )

import numpy as np
print( f"NumPy {np.__version__}\n" )

import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf
print( f"TensorFlow {tf.__version__}" )
print( f"tf.keras.backend.image_data_format() = {tf.keras.backend.image_data_format()}" )

# Count the number of GPUs as detected by tensorflow
gpus = tf.config.list_physical_devices('GPU')
print( f"TensorFlow detected { len(gpus) } GPU(s):" )
for i, gpu in enumerate(gpus):
  print( f".... GPU No. {i}: Name = {gpu.name} , Type = {gpu.device_type}" )

Python 3.8.16 (default, Dec  7 2022, 01:12:13) 
[GCC 7.5.0]

NumPy 1.21.6

TensorFlow 2.9.2
tf.keras.backend.image_data_format() = channels_last
TensorFlow detected 1 GPU(s):
.... GPU No. 0: Name = /physical_device:GPU:0 , Type = GPU


Install pytesseract

In [3]:
!sudo apt install tesseract-ocr
!pip install pytesseract
!pip install -q transformers datasets
!pip install -q pytorch-lightning wandb

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 20 not upgraded.
Need to get 4,795 kB of archives.
After this operation, 15.8 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-eng all 4.00~git24-0e00fe6-1.2 [1,588 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-osd all 4.00~git24-0e00fe6-1.2 [2,989 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr amd64 4.00~git2288-10f4998a-2 [218 kB]
Fetched 4,795 kB in 0s (12.3 MB/s)
debconf: unable to initia

     |████████████████████████████████| 5.8 MB 4.7 MB/s 
     |████████████████████████████████| 452 kB 84.7 MB/s 
     |████████████████████████████████| 7.6 MB 78.8 MB/s 
     |████████████████████████████████| 182 kB 89.0 MB/s 
     |████████████████████████████████| 212 kB 101.4 MB/s 
     |████████████████████████████████| 132 kB 91.6 MB/s 
     |████████████████████████████████| 127 kB 75.8 MB/s 
     |████████████████████████████████| 800 kB 4.6 MB/s 
     |████████████████████████████████| 1.9 MB 85.9 MB/s 
     |████████████████████████████████| 512 kB 95.7 MB/s 
     |████████████████████████████████| 125 kB 99.8 MB/s 
     |████████████████████████████████| 182 kB 90.0 MB/s 
     |████████████████████████████████| 174 kB 79.4 MB/s 
     |████████████████████████████████| 62 kB 1.4 MB/s 
     |████████████████████████████████| 173 kB 91.4 MB/s 
     |████████████████████████████████| 168 kB 87.4 MB/s 
     |████████████████████████████████| 168 kB 88.7 MB/s 
     |███████████

Reset runtime

In [ ]:
import os
os.kill(os.getpid(), 9)

Import library

In [3]:
import cv2
import pytesseract
import shutil
import os
import random
from PIL import Image

import re
import csv
import os

import pyarrow as pa
from datasets import DatasetDict
from datasets.arrow_dataset import Dataset

Mount drive

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
import re
import csv
import os
import pyarrow as pa
import torch
import pyarrow as pa
from datasets import DatasetDict
from datasets.arrow_dataset import Dataset

## Create Data from Chart2text data

In [174]:
List_key = []
# Open the CSV file for reading
for i in range(1000): 
  with open('/content/drive/MyDrive/DADS7202_final/Chart2text/data/'+str(i)+'.csv', 'r') as d:
 # Read the contents of the file into a list of rows
    rows = list(csv.reader(d))
# Concatenate the rows into a single string
    csv_string = '\n'.join(['\t'.join(row) for row in rows])
    csv_string = re.sub('\s+',' ',csv_string)
    List_key.append(csv_string)
len(List_key)

1000

In [175]:
import pickle

with open('List_key_0to1000.pkl', 'wb') as f:
    pickle.dump(List_key, f)

Load

In [7]:
import pickle

with open('List_key_0to2000.pkl', 'rb') as f:
    List_key = pickle.load(f)

In [8]:
len(List_key)

2000

In [178]:
txt_key = []
for i in range(1000): 
 f = open('/content/drive/MyDrive/DADS7202_final/Chart2text/captions/'+str(i)+'.txt', 'r')
 content = f.read()
 content = re.sub('\s+',' ',content)
 txt_key.append(content)
len(txt_key)

1000

In [179]:
import pickle

with open('txt_key_0to1000.pkl', 'wb') as f:
    pickle.dump(txt_key, f)

Load

In [9]:
import pickle

with open('txt_key_0to2000.pkl', 'rb') as f:
    txt_key = pickle.load(f)

In [10]:
print(List_key[0])

Quarter Number of users in millions Q4 '19 2498 Q3 '19 2449 Q2 '19 2414 Q1 '19 2375 Q4 '18 2320 Q3 '18 2271 Q2 '18 2234 Q1 '18 2196 Q4 '17 2129 Q3 '17 2072 Q2 '17 2006 Q1 '17 1936 Q4 '16 1860 Q3 '16 1788 Q2 '16 1712 Q1 '16 1654 Q4 '15 1591 Q3 '15 1545 Q2 '15 1490 Q1 '15 1441 Q4 '14 1393 Q3 '14 1350 Q2 '14 1317 Q1 '14 1276 Q4 '13 1228 Q3 '13 1189 Q2 '13 1155 Q1 '13 1110 Q4 '12 1056 Q3 '12 1007 Q2 '12 955 Q1 '12 901 Q4 '11 845 Q3 '11 800 Q2 '11 739 Q1 '11 680 Q4 '10 608 Q3 '10 550 Q2 '10 482 Q1 '10 431 Q4 '09 360 Q3 '09 305 Q2 '09 242 Q1 '09 197 Q3 '08 100


In [11]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [12]:
# Create a list of lists that represents the data
train_data = [List_key[:1400] ,txt_key[:1400]]

# Create a list of pyarrow.Array objects from the data
arrays_train = [pa.array(col) for col in train_data]

# Create a pyarrow Table object from the arrays
table1 = pa.Table.from_arrays(arrays_train, names=['key', 'text'])
#print(type(table1))

# Create a Dataset object from the table
dataset1 = Dataset(table1)
#print(type(dataset1))

# Create a list of lists that represents the data
val_data = [List_key[1400:1700] ,txt_key[1400:1700]]

# Create a list of pyarrow.Array objects from the data
arrays_val = [pa.array(col) for col in val_data]

# Create a pyarrow Table object from the arrays
table2 = pa.Table.from_arrays(arrays_val, names=['key', 'text'])
#print(type(table1))

# Create a Dataset object from the table
dataset2 = Dataset(table2)
print(type(dataset2))

# Create a list of lists that represents the data
test_data = [List_key[1700:] ,txt_key[1700:]]

# Create a list of pyarrow.Array objects from the data
arrays_test = [pa.array(col) for col in test_data]

# Create a pyarrow Table object from the arrays
table3 = pa.Table.from_arrays(arrays_test, names=['key', 'text'])
#print(type(table1))

# Create a Dataset object from the table
dataset3 = Dataset(table3)
print(type(dataset3))

# Create a dictionary that maps keys to the Dataset objects
dataset = {
    'train': dataset1,
    'validation': dataset2,
    'test': dataset3
}

# Access a Dataset object using its key
print(dataset)  # Output: Dataset object with train_data
print(type(dataset))
#print(dataset['validation'])  # Output: Dataset object with validation_data
#print(dataset['test'])  # Output: Dataset object with test_data

<class 'datasets.arrow_dataset.Dataset'>
<class 'datasets.arrow_dataset.Dataset'>
{'train': Dataset({
    features: ['key', 'text'],
    num_rows: 1400
}), 'validation': Dataset({
    features: ['key', 'text'],
    num_rows: 300
}), 'test': Dataset({
    features: ['key', 'text'],
    num_rows: 300
})}
<class 'dict'>


In [13]:
from datasets import DatasetDict

dataset_dict = DatasetDict(dataset)
print(type(dataset_dict))
print(dataset_dict)

<class 'datasets.dataset_dict.DatasetDict'>
DatasetDict({
    train: Dataset({
        features: ['key', 'text'],
        num_rows: 1400
    })
    validation: Dataset({
        features: ['key', 'text'],
        num_rows: 300
    })
    test: Dataset({
        features: ['key', 'text'],
        num_rows: 300
    })
})


In [14]:
print(dataset['train'][1])

{'key': "NBA player Number of points scored Kareem Abdul-Jabbar 38387 Karl Malone 36928 LeBron James* 33655 Kobe Bryant 33643 Michael Jordan 32292 Dirk Nowitzki 31560 Wilt Chamberlain 31419 Shaquille O'Neal 28596 Moses Malone 27409 Elvin Hayes 27313 Hakeem Olajuwon 26946 Oscar Robertson 26710 Dominique Wilkins 26668 Tim Duncan 26496 Paul Pierce 26397 John Havlicek 26395 Kevin Garnett 26071 Carmelo Anthony* 26067 Vince Carter* 25623 Alex English 25613 Reggie Miller 25279 Jerry West 25192 Patrick Ewing 24815 Ray Allen 24505 Allen Iverson 24368 Charles Barkley 23757 Robert Parish 23334 Adrian Dantley 23177 Dwyane Wade 23165 Elgin Baylor 23149 Kevin Durant* 22940", 'text': "How many points does LeBron James have ? In January 2020 , LeBron James passed Kobe Bryant on the NBA all-time scoring list , scoring 29 points to in a regular season game against the Philadelphia 76ers . If James continues to produce at about his career average , he has a shot to move up even further and pass Karl Malo

In [15]:
example = dataset_dict['train'][1]

print("Key:", example["key"])
print("Text:", example["text"])

Key: NBA player Number of points scored Kareem Abdul-Jabbar 38387 Karl Malone 36928 LeBron James* 33655 Kobe Bryant 33643 Michael Jordan 32292 Dirk Nowitzki 31560 Wilt Chamberlain 31419 Shaquille O'Neal 28596 Moses Malone 27409 Elvin Hayes 27313 Hakeem Olajuwon 26946 Oscar Robertson 26710 Dominique Wilkins 26668 Tim Duncan 26496 Paul Pierce 26397 John Havlicek 26395 Kevin Garnett 26071 Carmelo Anthony* 26067 Vince Carter* 25623 Alex English 25613 Reggie Miller 25279 Jerry West 25192 Patrick Ewing 24815 Ray Allen 24505 Allen Iverson 24368 Charles Barkley 23757 Robert Parish 23334 Adrian Dantley 23177 Dwyane Wade 23165 Elgin Baylor 23149 Kevin Durant* 22940
Text: How many points does LeBron James have ? In January 2020 , LeBron James passed Kobe Bryant on the NBA all-time scoring list , scoring 29 points to in a regular season game against the Philadelphia 76ers . If James continues to produce at about his career average , he has a shot to move up even further and pass Karl Malone with a

In [16]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [17]:
from transformers import AutoTokenizer, AutoModelWithLMHead, AutoModelForSeq2SeqLM

*************** เปลี่ยน tokenizer เป็นตาม huggingface เลือก model ****************

In [18]:
tokenizer = AutoTokenizer.from_pretrained("t5-base")  ## edit here

prefix = "Text generation from keyword: "
max_input_length = 512
max_target_length = 256

def preprocess_examples(examples):
  # encode the code-docstring pairs
  codes = examples['key']
  docstrings = examples['text']
  
  inputs = [prefix + code for code in codes]
  model_inputs = tokenizer(inputs, max_length=max_input_length, padding="max_length", truncation=True)

  # encode the summaries
  labels = tokenizer(docstrings, max_length=max_target_length, padding="max_length", truncation=True).input_ids

  # important: we need to replace the index of the padding tokens by -100
  # such that they are not taken into account by the CrossEntropyLoss
  labels_with_ignore_index = []
  for labels_example in labels:
    labels_example = [label if label != 0 else -100 for label in labels_example]
    labels_with_ignore_index.append(labels_example)
  
  model_inputs["labels"] = labels_with_ignore_index

  return model_inputs

Downloading:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

/usr/local/lib/python3.8/dist-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [19]:
dataset = dataset_dict.map(preprocess_examples, batched=True)

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [20]:
from torch.utils.data import DataLoader

dataset.set_format(type="torch", columns=['input_ids', 'attention_mask', 'labels'])
train_dataloader = DataLoader(dataset['train'], shuffle=True, batch_size=8)
valid_dataloader = DataLoader(dataset['validation'], batch_size=4)
test_dataloader = DataLoader(dataset['test'], batch_size=4)

In [21]:
batch = next(iter(train_dataloader))
print(batch.keys())

dict_keys(['input_ids', 'attention_mask', 'labels'])


In [22]:
!pip install -q pytorch-lightning wandb

******* ปรับ epoch กับ model เป็นตาม huggingface **********

### Create CodeT5 (model1)

In [23]:
from transformers import T5ForConditionalGeneration, AdamW, get_linear_schedule_with_warmup
import pytorch_lightning as pl

class CodeT5(pl.LightningModule):
    def __init__(self, lr=5e-5, num_train_epochs=100, warmup_steps=100):  ## edit here
        super().__init__()
        self.model = AutoModelForSeq2SeqLM.from_pretrained("t5-base")  ## edit here
        self.save_hyperparameters()

    def forward(self, input_ids, attention_mask, labels=None):     
        outputs = self.model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        return outputs
    
    def common_step(self, batch, batch_idx):
        outputs = self(**batch)
        loss = outputs.loss

        return loss
      
    def training_step(self, batch, batch_idx):
        loss = self.common_step(batch, batch_idx)     
        # logs metrics for each training_step,
        # and the average across the epoch
        self.log("training_loss", loss)

        return loss

    def validation_step(self, batch, batch_idx):
        loss = self.common_step(batch, batch_idx)     
        self.log("validation_loss", loss, on_epoch=True)

        return loss

    def test_step(self, batch, batch_idx):
        loss = self.common_step(batch, batch_idx)     

        return loss

    def configure_optimizers(self):
        # create optimizer
        optimizer = AdamW(self.parameters(), lr=self.hparams.lr)
        # create learning rate scheduler
        num_train_optimization_steps = self.hparams.num_train_epochs * len(train_dataloader)
        lr_scheduler = {'scheduler': get_linear_schedule_with_warmup(optimizer,
                                                    num_warmup_steps=self.hparams.warmup_steps,
                                                    num_training_steps=num_train_optimization_steps),
                        'name': 'learning_rate',
                        'interval':'step',
                        'frequency': 1}
        
        return {"optimizer": optimizer, "lr_scheduler": lr_scheduler}

    def train_dataloader(self):
        return train_dataloader

    def val_dataloader(self):
        return valid_dataloader

    def test_dataloader(self):
        return test_dataloader

In [24]:
import wandb

wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [25]:
tokenizer.decode(batch['input_ids'][0])

'Text generation from keyword: Fiscal year Total number of stores 2018 1844 2017 1822 2016 1802 2015 1792 2014 1790 2013 1793 2012 1778 2011 1763 2010 1750 2009 1740 2008 1682 2007 1591 2006 1488</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><

In [26]:
labels = batch['labels'][0]
tokenizer.decode([label for label in labels if label != -100])

"Target had a total of 1,844 stores open throughout North America as of 2018. Target Corporation operates a chain of general merchandise stores, which offer a wide variety of general merchandise and food products to their customers. The company has operated primarily in the United States since its inception. Target Corporation is one of the most valuable retail brands in the world. The development of an American retail giant The company started out as the Dayton Dry Goods Company in 1902, later changing its name to the Dayton Company, but more commonly known as Dayton's. The company was run under the Dayton-Hudson Corporation banner up until the year 2000, when it was renamed Target Corporation. The company spread across the United States and even entered the Canadian market for a brief period, but all of the company's Canadian stores were closed in 2015. In 2018, Target had revenues amounting to approximately 75 billion U.S. dollars, making it one of the leading American retailers. Wh

In [27]:
model = CodeT5()

Downloading:   0%|          | 0.00/892M [00:00<?, ?B/s]

### Train (fit) model1

***** ถ้าใช้ k2t ให้ #logger=wandb_logger ด้วย **********

In [74]:
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor

import sys
import signal

save_directory = "/content/drive/MyDrive/DADS7202_final/model1-t5base-2000d-100e-256-v1-timeout"

def save_and_exit(signal, frame):
    model.model.save_pretrained(save_directory)
    sys.exit(0)

signal.signal(signal.SIGINT, save_and_exit)


wandb_logger = WandbLogger(name='codet5-finetune-code-summarization-ruby-shuffle', project='CodeT5')
# for early stopping, see https://pytorch-lightning.readthedocs.io/en/1.0.0/early_stopping.html?highlight=early%20stopping
early_stop_callback = EarlyStopping(
    monitor='validation_loss',
    patience=10,
    strict=False,
    verbose=False,
    mode='min'
)
lr_monitor = LearningRateMonitor(logging_interval='step')

trainer = Trainer(gpus=1, 
                  default_root_dir="/content/drive/MyDrive/CodeT5/Notebooks/Checkpoints", 
                  logger=wandb_logger, 
                  callbacks=[early_stop_callback, lr_monitor])
trainer.fit(model)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 
-----------------------------------------------------
222 M     Trainable params
0         Non-trainable params
222 M     Total params
891.614   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

#### check layers and weight in model

In [60]:
# Access the state_dict attribute to get a dictionary of weights
weights = model.state_dict()

# Print the keys and shapes of the weights
#for key, value in weights.items():
    #print(key, value.shape)

In [ ]:
# Get the weights of the first layer
layer_weights = weights['model.decoder.final_layer_norm.weight']

# Print the shape of the weights
print(layer_weights)

************* ตั้ง path ที่จะ save model อันแรก ***********

In [28]:
save_directory = "/content/drive/MyDrive/DADS7202_final/model1-t5base-2000d-100e-256-v1"

In [75]:
# save in the current working directory, you can change this of course
model.model.save_pretrained(save_directory)

In [33]:
dataset2 = dataset_dict
print(dataset2['test'])

Dataset({
    features: ['key', 'text'],
    num_rows: 300
})


In [34]:
test_example = dataset2['test'][0]
print("Key:", test_example['key'])

Key: Year Attendance in millions 2013 59.44 2012 58.68 2011 54.3 2010 47.36 2009 50.42 2008 48.66 2007 49.51 2006 49.76 2005 51.82 2004 50.74 2003 48.58


In [35]:
test_example = dataset2['test'][0]
print("Text:", test_example['text'])

Text: This graph depicts the total attendance at popular music concerts in the United States from 2003 to 2013 . In 2013 , 59.44 million people attended a popular music concert at least once . The music concert industry – additional information In 2016 , Beyonce and Guns ' N ' Rose were among the most successful music tours in North America , generating 169.4 million U.S. dollars and 130.8 million U.S. dollars , respectively in gross revenue . During that same time , Bruce Springsteen and Coldplay led some of the most successful global tours , generating 268.3 million U.S. dollars and 241 million U.S. dollars , respectively.Ticket sales revenue in North America reached their highest point in 2016 , totaling 7.3 billion U.S. dollars . The concert industry is benefiting from increasing number of shows , ticket sales , as well as ticket prices . Foro Sol , a large outdoor venue built in 1993 in Mexico City , had ticket sales amounting to almost one million units in 2016 . Live Nation is o

In [36]:
#from transformers import T5ForConditionalGeneration
#model = T5ForConditionalGeneration.from_pretrained(save_directory)

**************** AutoModelForSeq2SeqLM ของ แต่ละ model ให้ดูตาม huggingface ******************

In [37]:
model = AutoModelForSeq2SeqLM.from_pretrained(save_directory)

### Generate data from own model

In [38]:
# prepare for the model
input_ids = tokenizer(test_example['key'], return_tensors='pt').input_ids
# generate
outputs = model.generate(input_ids,min_length=150, max_length=200)
print("Generated :", tokenizer.decode(outputs[0], skip_special_tokens=True))

Generated : The number of people attending concerts in the United States has increased steadily over the past decade, reaching 59.44 million in 2013. This is a decrease from 2003, when the number of concerts in the United States amounted to 48.58 million. The number of concerts in the United States has also increased, reaching a record 59.44 million in 2013. The number of concerts in the United States has also increased, reaching almost a quarter of the total audience in 2013. The number of concerts in the United States has increased steadily over the past decade, reaching almost a quarter of the total audience in 2013. The number of concerts in the United States has also increased in recent years, reaching almost a quarter of the total number of concerts in the United States in 2013. In 2013, the number of concerts in the United States was almost a quarter of the total number of concerts


Create list of predict data and ref data

In [40]:
predict = []
refer = []
for i in range(300):
  test_example = dataset2['test'][i]
  input_ids = tokenizer(test_example['key'], return_tensors='pt').input_ids
  outputs = model.generate(input_ids,min_length=150, max_length=200)
  predict.append(tokenizer.decode(outputs[0], skip_special_tokens=True))
  refer.append(test_example['text'])

In [41]:
print(len(predict))
print(len(refer))

300
300


In [47]:
## save list to txt
with open("predict.txt", "w") as output:
    output.write(str(predict))

with open("refer.txt", "w") as output:
    output.write(str(refer))

### Evaluation

In [42]:
pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 81 kB 3.9 MB/s 


In [43]:
import evaluate
bleu = evaluate.load("bleu")
results = bleu.compute(predictions=predict, references=refer)
print(results)

{'bleu': 0.08371896898399186, 'precisions': [0.29494045743809466, 0.10396719717625177, 0.05211324527258418, 0.03074090724209535], 'brevity_penalty': 1.0, 'length_ratio': 1.1108181881809487, 'translation_length': 47613, 'reference_length': 42863}


In [44]:
pip install rouge

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [45]:
from rouge import Rouge 

hypothesis = "the #### transcript is a written version of each day 's cnn student news program use this transcript to he    lp students with reading comprehension and vocabulary use the weekly newsquiz to test your knowledge of storie s you     saw on cnn student news"

reference = "this page includes the show transcript use the transcript to help students with reading comprehension and     vocabulary at the bottom of the page , comment for a chance to be mentioned on cnn student news . you must be a teac    her or a student age # # or older to request a mention on the cnn student news roll call . the weekly newsquiz tests     students ' knowledge of even ts in the news"

rouge = Rouge()
scores = rouge.get_scores(predict, refer, avg = True)
print(scores)

{'rouge-1': {'r': 0.29698571043181826, 'p': 0.4118434423796116, 'f': 0.3084603097999853}, 'rouge-2': {'r': 0.12310709187027363, 'p': 0.14709547472175333, 'f': 0.11355072886626777}, 'rouge-l': {'r': 0.28348515926642565, 'p': 0.39247243362288253, 'f': 0.2939980927651744}}


## Create data from Ielt DATASET

*********** แก้ path ************

In [ ]:
List_key = []
for i in range(len(os.listdir('/content/drive/MyDrive/DADS7202_final/IELTS/table'))): 
  src = '/content/drive/MyDrive/DADS7202_final/IELTS/table/'+str(i)+'.png'
  img = cv2.imread(src)
  # Adding custom options
  custom_config = r'--psm 1'
  text = pytesseract.image_to_string(img, config=custom_config)
  #print(text)
  text_split = text.split('\n')
  #print(text_split)
  while '' in text_split:
    text_split.remove('')
  while ' ' in text_split:
    text_split.remove(' ')
  key = ' '.join(text_split[:-2])
  List_key.append(key)
len(List_key)

In [54]:
import pickle

with open('List_key_ielts.pkl', 'wb') as f:
    pickle.dump(List_key, f)

Load ielts key

In [48]:
import pickle

with open('List_key_ielts.pkl', 'rb') as f:
    List_key = pickle.load(f)

In [130]:
txt_key = []
for i in range(len(os.listdir('/content/drive/MyDrive/DADS7202_final/IELTS/captions'))): 
  f = open('/content/drive/MyDrive/DADS7202_final/IELTS/captions/'+str(i)+'.txt', 'r')
  content = f.read()
  content = re.sub('\s+',' ',content)
  txt_key.append(content)
len(txt_key)

201

In [58]:
import pickle

with open('txt_key_ielts.pkl', 'wb') as f:
    pickle.dump(txt_key, f)

Load ielts txt

In [49]:
import pickle

with open('txt_key_ielts.pkl', 'rb') as f:
    txt_key = pickle.load(f)

In [50]:
import pyarrow as pa

# Create a list of lists that represents the data
train_data = [List_key[:140] ,txt_key[:140]]

# Create a list of pyarrow.Array objects from the data
arrays_train = [pa.array(col) for col in train_data]

# Create a pyarrow Table object from the arrays
table1 = pa.Table.from_arrays(arrays_train, names=['key', 'text'])
#print(type(table1))

# Create a Dataset object from the table
dataset1 = Dataset(table1)
#print(type(dataset1))

# Create a list of lists that represents the data
val_data = [List_key[140:170] ,txt_key[140:170]]

# Create a list of pyarrow.Array objects from the data
arrays_val = [pa.array(col) for col in val_data]

# Create a pyarrow Table object from the arrays
table2 = pa.Table.from_arrays(arrays_val, names=['key', 'text'])
#print(type(table1))

# Create a Dataset object from the table
dataset2 = Dataset(table2)
print(type(dataset2))

# Create a list of lists that represents the data
test_data = [List_key[170:] ,txt_key[170:]]

# Create a list of pyarrow.Array objects from the data
arrays_test = [pa.array(col) for col in test_data]

# Create a pyarrow Table object from the arrays
table3 = pa.Table.from_arrays(arrays_test, names=['key', 'text'])
#print(type(table1))

# Create a Dataset object from the table
dataset3 = Dataset(table3)
print(type(dataset3))

# Create a dictionary that maps keys to the Dataset objects
ielts_dataset = {
    'train': dataset1,
    'validation': dataset2,
    'test': dataset3
}

# Access a Dataset object using its key
print(ielts_dataset)  # Output: Dataset object with train_data
print(type(ielts_dataset))
#print(dataset['validation'])  # Output: Dataset object with validation_data
#print(dataset['test'])  # Output: Dataset object with test_data

<class 'datasets.arrow_dataset.Dataset'>
<class 'datasets.arrow_dataset.Dataset'>
{'train': Dataset({
    features: ['key', 'text'],
    num_rows: 140
}), 'validation': Dataset({
    features: ['key', 'text'],
    num_rows: 30
}), 'test': Dataset({
    features: ['key', 'text'],
    num_rows: 31
})}
<class 'dict'>


In [51]:
print(ielts_dataset['train'][0])

{'key': 'Holiday leisure activities Number of hours |At the beach Reading books Chinese tourists 3 8 American tourists 4 5. Turkish tourists 5 4', 'text': 'The bar chart compares the amount of time each day that people from four different countries spent doing activities while on holiday in Greece in August 2019. The activities were going to the beach, visiting different attractions, and reading. Overall, the tourists spent the most time reading books while they spent the least time visiting places. The Chinese tourists spent the longest reading books with a total of eight hours per day. By contrast, the Brazilians spent the longest at the beach with a total of six hours. Both the American and the Turkish tourists spent the same number of hours every day visiting places with a total of three hours each. Similarly, the Turkish and Brazilian groups spent a total of four hours each reading books. The Chinese tourists spent twice the amount of time reading books as the Turkish and Brazilia

In [52]:
from datasets import DatasetDict

dataset_dict = DatasetDict(ielts_dataset)
print(type(dataset_dict))
print(dataset_dict)

<class 'datasets.dataset_dict.DatasetDict'>
DatasetDict({
    train: Dataset({
        features: ['key', 'text'],
        num_rows: 140
    })
    validation: Dataset({
        features: ['key', 'text'],
        num_rows: 30
    })
    test: Dataset({
        features: ['key', 'text'],
        num_rows: 31
    })
})


**Train ielts data**

In [53]:
from transformers import AutoTokenizer, AutoModelWithLMHead, AutoModelForSeq2SeqLM

*********** แก้ tokenizer ตาม model เรา **********

In [54]:
tokenizer = AutoTokenizer.from_pretrained("t5-base")

prefix = "Text generation from keyword: "
max_input_length = 512
max_target_length = 256

def preprocess_examples(examples):
  # encode the code-docstring pairs
  codes = examples['key']
  docstrings = examples['text']
  
  inputs = [prefix + code for code in codes]
  model_inputs = tokenizer(inputs, max_length=max_input_length, padding="max_length", truncation=True)

  # encode the summaries
  labels = tokenizer(docstrings, max_length=max_target_length, padding="max_length", truncation=True).input_ids

  # important: we need to replace the index of the padding tokens by -100
  # such that they are not taken into account by the CrossEntropyLoss
  labels_with_ignore_index = []
  for labels_example in labels:
    labels_example = [label if label != 0 else -100 for label in labels_example]
    labels_with_ignore_index.append(labels_example)
  
  model_inputs["labels"] = labels_with_ignore_index

  return model_inputs

/usr/local/lib/python3.8/dist-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [55]:
dataset = dataset_dict.map(preprocess_examples, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [56]:
from torch.utils.data import DataLoader

dataset.set_format(type="torch", columns=['input_ids', 'attention_mask', 'labels'])
train_dataloader = DataLoader(dataset['train'], shuffle=True, batch_size=8)
valid_dataloader = DataLoader(dataset['validation'], batch_size=4)
test_dataloader = DataLoader(dataset['test'], batch_size=4)

In [57]:
batch = next(iter(train_dataloader))
print(batch.keys())

dict_keys(['input_ids', 'attention_mask', 'labels'])


In [58]:
!pip install -q pytorch-lightning wandb

### Create CodeT5 (model2)

*********** แก้ AutoModelForSeq2SeqLM ตามแต่ละโมเดลตาม huggingface ********

********* แก้ epoch ***********

In [59]:
from transformers import T5ForConditionalGeneration, AdamW, get_linear_schedule_with_warmup
import pytorch_lightning as pl

class CodeT5(pl.LightningModule):
    def __init__(self, lr=5e-5, num_train_epochs=100, warmup_steps=100):  #edit here
        super().__init__()
        self.model = AutoModelForSeq2SeqLM.from_pretrained(save_directory)  #edit here
        self.save_hyperparameters()

    def forward(self, input_ids, attention_mask, labels=None):     
        outputs = self.model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        return outputs
    
    def common_step(self, batch, batch_idx):
        outputs = self(**batch)
        loss = outputs.loss

        return loss
      
    def training_step(self, batch, batch_idx):
        loss = self.common_step(batch, batch_idx)     
        # logs metrics for each training_step,
        # and the average across the epoch
        self.log("training_loss", loss)

        return loss

    def validation_step(self, batch, batch_idx):
        loss = self.common_step(batch, batch_idx)     
        self.log("validation_loss", loss, on_epoch=True)

        return loss

    def test_step(self, batch, batch_idx):
        loss = self.common_step(batch, batch_idx)     

        return loss

    def configure_optimizers(self):
        # create optimizer
        optimizer = AdamW(self.parameters(), lr=self.hparams.lr)
        # create learning rate scheduler
        num_train_optimization_steps = self.hparams.num_train_epochs * len(train_dataloader)
        lr_scheduler = {'scheduler': get_linear_schedule_with_warmup(optimizer,
                                                    num_warmup_steps=self.hparams.warmup_steps,
                                                    num_training_steps=num_train_optimization_steps),
                        'name': 'learning_rate',
                        'interval':'step',
                        'frequency': 1}
        
        return {"optimizer": optimizer, "lr_scheduler": lr_scheduler}

    def train_dataloader(self):
        return train_dataloader

    def val_dataloader(self):
        return valid_dataloader

    def test_dataloader(self):
        return test_dataloader

In [ ]:
import wandb

wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 

··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [60]:
tokenizer.decode(batch['input_ids'][0])

'Text generation from keyword: 515,600 Canadians aged between 25 and 35 and 213,400 Canadians aged between 45 and 55 hold a degree. Their main fields of study were. Percentage of main fields of study 25-35 years|45-55 years Administration 19% 15% Social Science, Arts 20% 21% Education 15% 13% Science, Maths, Computers 18% 10% Engineering 9% 19% Humanities 4% 5% Nursing 7% 11% MBA 6% 3%</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pa

In [61]:
labels = batch['labels'][0]
tokenizer.decode([label for label in labels if label != -100])

'These two pie charts show the differences between two groups of Canadians (25 to 35 and 45 to 55 year olds) in terms of their post-school qualifications. The first point to note is the huge increase in the number of people with qualifications. The younger group is more than two times bigger than the older group (515,600 compared to 213,400). Comparing the two groups, Humanities and Nursing have experienced an increase in popularity with older graduates; Nursing rose by 4%. However, the biggest gain was made by engineering, which increased from 9% to 19% of the whole. Some subjects were more popular with the younger age group. The biggest loss in graduate numbers were to Science, Maths and Computers which, as a group, have decreased by 8%. Administration has declined by 4%. To sum up, the graphs show a more than twofold increase in the number of graduates. The most noticeable change in subject percentages is the increase in the number of engineering graduates.</s>'

Create new model to train again

In [62]:
model = CodeT5()

### Train (fit) model2

In [63]:
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor

wandb_logger = WandbLogger(name='codet5-finetune-code-summarization-ruby-shuffle', project='CodeT5')
# for early stopping, see https://pytorch-lightning.readthedocs.io/en/1.0.0/early_stopping.html?highlight=early%20stopping
early_stop_callback = EarlyStopping(
    monitor='validation_loss',
    patience=10,
    strict=False,
    verbose=False,
    mode='min'
)
lr_monitor = LearningRateMonitor(logging_interval='step')

trainer = Trainer(gpus=1, 
                  default_root_dir="/content/drive/MyDrive/CodeT5/Notebooks/Checkpoints", 
                  #logger=wandb_logger, 
                  callbacks=[early_stop_callback, lr_monitor])
trainer.fit(model)

wandb: Currently logged in as: srichaiperksiv (dads7202). Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.8/dist-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:441: LightningDeprecationWarning: Setting `Trainer(gpus=1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=1)` instead.
  rank_zero_deprecation(
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/local/lib/python3.8/dist-packages/pytorch_lightning/loops/utilities.py:94: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
  rank_zero_warn(
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:30

Sanity Checking: 0it [00:00, ?it/s]

/usr/local/lib/python3.8/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/usr/local/lib/python3.8/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/usr/local/lib/python3.8/dist-packages/pytorch_lightning/trainer/trainer.py:1595: PossibleUserWarning: The number of training batches (18) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower val

Training: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

******* set new directory ที่จะsaveใน model อันใหม่หลัง train กับ ielts แล้ว ********

In [64]:
save_directory = "/content/drive/MyDrive/DADS7202_final/model2-t5base-2000d-100e-256-v1"

In [65]:
 # save in the current working directory, you can change this of course
model.model.save_pretrained(save_directory)

In [66]:
dataset2 = dataset_dict
print(dataset2['test'])

Dataset({
    features: ['key', 'text'],
    num_rows: 31
})


In [67]:
test_example = dataset2['test'][0]
print("Key:", test_example['key'])

Key: yearly income year Amandline Bakesry Mari Bakeshop Bolo Cakery 2001 20,000.00 80,000.00 58,000.00 2002 25,000.00 95,000.00 58,000.00 2003 21,000.00 90,000.00 58,000.00 2004 39,000.00 80,000.00 58,000.00 2005 39,000.00 92,000.00 58,000.00 2006 38,000.00 61,000.00 60,000.00 2007 41,500.00 59,000.00 66,000.00 2008 62,000.00 60,000.00 97,000.00 2009 62,500.00 41,000.00 105,000.00


In [68]:
test_example = dataset2['test'][0]
print("Text:", test_example['text'])

Text: The graph shows information about the amount of money which was earned by three bakeries in Calgary, over a ten-year period between 2000 and 2010. Overall, what stands out from the graph is that there were considerable upward trends in the income of both Bolo Cakery and Amandine Bakery, while the earnings of Mari Bakeshop saw a considerable fall over the period in question. Another interesting point is that Mari Bakeshop was the most popular bakery in 2000, but in 2010, Amandine Bakery earned more money than the others. Looking at the details, as regards Bolo Cakery, income started at $20,000 in 2000, then there was a fluctuation over the next three years, at which point it levelled off at just under $40,000 until 2006. Then the figure went up significantly, finishing at around $65,000 in 2010. If we look at Amandine Bakery, the trend was similar. Having remained stable at approximately $55,000 in the first half of the decade, income then rose sharply, reaching nearly $100,000 in

******** แก้ AutoModelForSeq2SeqLM ตาม huggingface

### Generate data from ielts model

In [69]:
model = AutoModelForSeq2SeqLM.from_pretrained(save_directory)

In [70]:
# prepare for the model
input_ids = tokenizer(test_example['key'], return_tensors='pt').input_ids
# generate
outputs = model.generate(input_ids,min_length=150, max_length=200)
print("Generated :", tokenizer.decode(outputs[0], skip_special_tokens=True))

Generated : The annual income of Amandline Bakesry in Bolo Cakery increased from 20,000.00 to 58,000.00 in 2001 and was steady in both years. In 2001, the income of the business increased from 20,000.00 to 20,000.00. In 2009, the business increased from 60,000.00 to 60,000.00. In 2009, the business increased from 60,000.00 to 60,000.00. In 2001, the business increased from 20,000.00 to 20,000.00. In 2000, the business increased from 20,000.00 to 20,000.00. In 2000, the business grew from 20,000.00 to 60,000.00. In 2003, the business grew from 25,000.00 to 500,000.00. In 2003, the business grew from 25,000.00 to 500,000.00. In 2008, the business grew from 60,000.00 to 60,000.00. In 2009, the business grew from 60,000.00 to 60,000.00. In 2008, the business grew from


### Evaluation

In [71]:
predict = []
refer = []
for i in range(31):
  test_example = dataset2['test'][i]
  input_ids = tokenizer(test_example['key'], return_tensors='pt').input_ids
  outputs = model.generate(input_ids,min_length=150, max_length=200)
  predict.append(tokenizer.decode(outputs[0], skip_special_tokens=True))
  refer.append(test_example['text'])

In [72]:
print(len(predict))
print(len(refer))

31
31


In [78]:
i = 10
print(predict[i])
print(refer[i])

The Burnaby Public Library has a large number of books read each year. In 2011 and 2012, the library had a record number of books read by men. In 2012, the number of books read by women was a record number, with men reading more than twice as many books as men. In 2012, the number of books read by women was a record number, with men reading more than twice as many books as men. In 2012, the number of books read by women was a record number, with men reading more than twice as many as women. In 2014, the number of books read by women was a record number, with women reading more than twice as many books as men. In 2012, the number of books read by women was a record number, with men reading more than twice as many as men. In 2012, the number of books read by women was a record number, with men reading more than twice as many as women
The graph gives information about Burnaby Public Library between 2011 and 2014. It shows how many library books people read over this four-year period. As c

In [ ]:
'''## save list to txt
with open("predict.txt", "w") as output:
    output.write(str(predict))

with open("refer.txt", "w") as output:
    output.write(str(refer))'''

In [ ]:
#pip install evaluate

In [73]:
import evaluate
bleu = evaluate.load("bleu")
results = bleu.compute(predictions=predict, references=refer)
print(results)

{'bleu': 0.053016217270575865, 'precisions': [0.327124773960217, 0.07855973813420622, 0.026335040234089245, 0.011955122310097481], 'brevity_penalty': 0.9940503195956194, 'length_ratio': 0.9940679489484091, 'translation_length': 5530, 'reference_length': 5563}


In [74]:
#pip install rouge

In [75]:
from rouge import Rouge 

hypothesis = "the #### transcript is a written version of each day 's cnn student news program use this transcript to he    lp students with reading comprehension and vocabulary use the weekly newsquiz to test your knowledge of storie s you     saw on cnn student news"

reference = "this page includes the show transcript use the transcript to help students with reading comprehension and     vocabulary at the bottom of the page , comment for a chance to be mentioned on cnn student news . you must be a teac    her or a student age # # or older to request a mention on the cnn student news roll call . the weekly newsquiz tests     students ' knowledge of even ts in the news"

rouge = Rouge()
scores = rouge.get_scores(predict, refer, avg = True)
print(scores)

{'rouge-1': {'r': 0.1921328436804615, 'p': 0.5225588236454283, 'f': 0.2747487223418793}, 'rouge-2': {'r': 0.05253037461480547, 'p': 0.15063226879799035, 'f': 0.07578468084015537}, 'rouge-l': {'r': 0.17998529340518846, 'p': 0.4877316455539334, 'f': 0.2570599391658229}}
